In [1]:
import os
import sys
# sys.path.append("../virtuosoNet")
# sys.path.append("C:\\Users\\fosca\\Desktop\\CNAM\\midi_beat_tracking\\FMP_0.1.0")

from pathlib import Path
import pandas as pd
import music21 as m21
import pretty_midi as pm
from  matplotlib import pyplot as plt
# import LibFMP.B
# import LibFMP.C2
# import LibFMP.C6
%matplotlib inline
import numpy as np
import math
import pandas as pd
import librosa
import soundfile

import IPython.display as ipd

In [39]:
def cleaned_m2m_df(midi2midi_path):
    m2mdf = pd.read_csv(midi2midi_path,sep="\t",skiprows=1,header=None, index_col=False)
    m2mdf.columns =["alignID", "alignOntime", "alignSitch" ,"alignPitch", "alignOnvel", "refID", "refOntime", "refSitch", "refPitch", "refOnvel","empty"]
    #drop the last column "empty"
    m2mdf = m2mdf.drop(columns=["empty"])
    #drop the rows where the alignOntime is -1 (note not existing in both files midi)
    return m2mdf[m2mdf["alignOntime"] != -1]

def generate_performance_annotations(m2m_df, cl_quant_ann):
    #create the list with the time of all annotations
    quant_ann_time_list = cl_quant_ann["time"].tolist()
    quant_ann_type_list = cl_quant_ann["type"].tolist()
    perf_ann_list = []
    for i,at in enumerate(quant_ann_time_list):
        p_ons=list(m2m_df[(m2m_df["refOntime"]>at-0.020) & (m2m_df["refOntime"]<at+0.020)]["alignOntime"]) #40 ms accepted window
        if len(p_ons) == 0: #no note matching the ann position in the performance
            perf_ann_list.append(None)
            print("missing corrispondency at quant time", at)
        else:
            perf_ann_list.append(np.mean(p_ons))

    # clean the values that are smaller than the value before
    for ind,at in enumerate(perf_ann_list[1:]):
        if (ind >0) and (at is not None) and (perf_ann_list[ind-1] is not None) and (at < perf_ann_list[ind-1]) :
            print("Unexpected value",db,"smaller than the previous",perf_ann_list[ind-1])
            perf_ann_list[ind] = None
    
    #save the index for the none values
    none_indices = [index for index,value in enumerate(perf_ann_list) if value is None]
     
    #check if there are no None at the end (there should no, because of the manual corrections)
    assert(perf_ann_list[-1] is not None)
    #interpolate to fill the None values in the middle
    filled_at = pd.Series(perf_ann_list).interpolate().tolist()
    #fill the values at the beginning
    ind = 0
    while math.isnan(filled_at[ind]): #find the number of None at the beginning
        ind +=1
    for i in range(ind)[::-1]: #fill, starting from the later one
        proposed_value = filled_at[i+1] - (filled_at[i+2]-filled_at[i+1])
        if proposed_value<0: #cap at 0
            proposed_value = 0
        filled_at[i] = proposed_value
    
    #associate the type of annotation
    assert(len(quant_ann_type_list) == len(filled_at))
    perd_ann_dict = dict(zip(filled_at, quant_ann_type_list))
    #add the Warning for the None values
    warnings = False
    for val in none_indices:
        perd_ann_dict[filled_at[val]] += "W"
        warnings = True
    
    return perd_ann_dict ,warnings
    
def beat_downbeat_to_text(annotations,out_path):
    path = Path(out_path)
    #produce the txt
    file_content = ""
    for a in annotations:
        file_content+= str(a) + "\t" + str(a) + "\t" + annotations[a] + "\n"
    with open(out_path,"w") as f:
        f.write(file_content)
        
def annotations_to_audio(midi_path, annotations, out_path):
    midi_data = pm.PrettyMIDI(str(midi_path))
    audio_fs = 6000
    beat = [a for a in annotations if annotations[a][0] == "b"]
    downbeat = [a for a in annotations if annotations[a][0] == "d"]
    midi_audio = midi_data.synthesize(fs=audio_fs)
    audio_beat = librosa.clicks(beat, sr=audio_fs, click_freq=900, length=len(midi_audio))
    audio_downbeat = librosa.clicks(downbeat, sr=audio_fs, click_freq=1200, length=len(midi_audio))
    soundfile.write(out_path,audio_beat + audio_downbeat+ midi_audio,audio_fs)
    

def perform_row_to_annotations(row):
    print("Processing opus", row.folder)
    #load the quantized annotations
    ann_quant_df = pd.read_csv(Path(row.folder,"ann_quant_cleaned.txt"),header=None, names=["time","time2","type"],sep='\t')
    #load the m2m alignment
    m2m = cleaned_m2m_df(row["midi2midi_alignment_path"])
    #generate the annotations
    ann_dict,warnings = generate_performance_annotations(m2m, ann_quant_df)
    #delete the old audio and ann
    old_audio = Path(row.folder,row.performer + "_click.wav")
    old_ann = Path(row.folder,row.performer + "_ann.txt")
    if old_audio.exists():
        old_audio.unlink()
    if old_ann.exists():
        old_ann.unlink()
    #write the annotations to txt
    beat_downbeat_to_text(ann_dict,Path(row.performed_midi_path[:-4] + "_ann.txt"))
    #write the audio
    annotations_to_audio(str(Path(row.performed_midi_path)),ann_dict,Path(row.performed_midi_path[:-4] + "_click.wav"))
    print("done")
    return warnings

In [48]:
df = pd.read_pickle("performance_dataframe_v2(bach).pkl")
# df = df[df["maestro_path"].notna()]
df.loc[0]

author                                                                   Bach
folder                                                     Bach/Fugue/bwv_846
midi2midi_alignment_path                Bach/Fugue/bwv_846/Shi05M_corresp.txt
performed_midi_path                             Bach/Fugue/bwv_846/Shi05M.mid
performer                                                               Shi05
score2midi_alignment                 Bach/Fugue/bwv_846/Shi05_infer_match.txt
score_midi_path                           Bach/Fugue/bwv_846/midi_cleaned.mid
score_xml_path                   Bach/Fugue/bwv_846/musicxml_cleaned.musicxml
title                                                           Fugue_bwv_846
maestro_path                2006/MIDI-Unprocessed_19_R1_2006_01-07_ORIG_MI...
maestro_alignment                                  (144.06770833333331, None)
Name: 0, dtype: object

## Bach

In [40]:
bach_df = df[df["author"]== "Bach"]
warnings = bach_df.apply(perform_row_to_annotations, axis = 1)

Processing opus Bach/Fugue/bwv_846
missing corrispondency at quant time 2.5
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_848
done
Processing opus Bach/Fugue/bwv_854
missing corrispondency at quant time 1.5
done
Processing opus Bach/Fugue/bwv_854
missing corrispondency at quant time 1.5
done
Processing opus Bach/Fugue/bwv_854
missing corrispondency at quant time 1.5
done
Processing opus Bach/Fugue/bwv_854
missing corrispondency at quant time 1.5
done
Processing opus Bach/Fugue/bwv_854
missing corrispondency at quant time 1.5
done
Processing opus Bach/Fugue/bwv_856
done
Processing opus Bach/Fugue/bwv_857
missing corrispondency at quant time 5.5
missing corrispondency at quant time 1

done
Processing opus Bach/Fugue/bwv_887
done
Processing opus Bach/Fugue/bwv_888
missing corrispondency at quant time 1.5
missing corrispondency at quant time 2.0
done
Processing opus Bach/Fugue/bwv_889
missing corrispondency at quant time 2.5
missing corrispondency at quant time 3.0
done
Processing opus Bach/Fugue/bwv_889
missing corrispondency at quant time 2.5
missing corrispondency at quant time 3.0
done
Processing opus Bach/Fugue/bwv_889
missing corrispondency at quant time 2.5
missing corrispondency at quant time 3.0
done
Processing opus Bach/Fugue/bwv_889
missing corrispondency at quant time 2.5
missing corrispondency at quant time 3.0
done
Processing opus Bach/Fugue/bwv_889
missing corrispondency at quant time 2.5
missing corrispondency at quant time 3.0
done
Processing opus Bach/Fugue/bwv_889
missing corrispondency at quant time 2.5
missing corrispondency at quant time 3.0
done
Processing opus Bach/Fugue/bwv_891
missing corrispondency at quant time 2.0
missing corrispondency at

done


In [41]:
warnings

0       True
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10      True
11      True
12      True
13      True
14      True
15     False
16      True
17      True
18      True
19      True
20      True
21      True
22      True
23      True
24      True
25     False
26     False
27      True
28     False
29     False
       ...  
142     True
143    False
144    False
145    False
146     True
147     True
148     True
149     True
150    False
151    False
152    False
153    False
154    False
155     True
156     True
157    False
158    False
159    False
160    False
161    False
162    False
163    False
164    False
165    False
166    False
167    False
168    False
169     True
170     True
171     True
Length: 172, dtype: bool

In [42]:
# create and save as excel for collaboration (also the not maestro aligned pieces)
df = pd.read_pickle("performance_dataframe_v2(bach).pkl")
bach_df = df[df["author"]== "Bach"]

bach_df_excel = bach_df[["author","performed_midi_path"]].copy()
bach_df_excel["done"] = [not e for e in list(warnings)]
bach_df_excel

# uncomment the following to write the file
bach_df_excel.to_excel("performance_bach.xlsx")

## Create for a single song

In [20]:
df[df["folder"]== "Bach/Prelude/bwv_883"] 

,author,folder,midi2midi_alignment_path,performed_midi_path,performer,score2midi_alignment,score_midi_path,score_xml_path,title
146,Bach,Bach/Prelude/bwv_883,Bach/Prelude/bwv_883/GuoE01_infer_corresp.txt,Bach/Prelude/bwv_883/GuoE01.mid,GuoE01,Bach/Prelude/bwv_883/GuoE01_infer_match.txt,Bach/Prelude/bwv_883/midi_cleaned.mid,Bach/Prelude/bwv_883/musicxml_cleaned.musicxml,Prelude_bwv_883
147,Bach,Bach/Prelude/bwv_883,Bach/Prelude/bwv_883/Herbert02_infer_corresp.txt,Bach/Prelude/bwv_883/Herbert02.mid,Herbert02,Bach/Prelude/bwv_883/Herbert02_infer_match.txt,Bach/Prelude/bwv_883/midi_cleaned.mid,Bach/Prelude/bwv_883/musicxml_cleaned.musicxml,Prelude_bwv_883
148,Bach,Bach/Prelude/bwv_883,Bach/Prelude/bwv_883/Lim01_infer_corresp.txt,Bach/Prelude/bwv_883/Lim01.mid,Lim01,Bach/Prelude/bwv_883/Lim01_infer_match.txt,Bach/Prelude/bwv_883/midi_cleaned.mid,Bach/Prelude/bwv_883/musicxml_cleaned.musicxml,Prelude_bwv_883
149,Bach,Bach/Prelude/bwv_883,Bach/Prelude/bwv_883/LuM01_infer_corresp.txt,Bach/Prelude/bwv_883/LuM01.mid,LuM01,Bach/Prelude/bwv_883/LuM01_infer_match.txt,Bach/Prelude/bwv_883/midi_cleaned.mid,Bach/Prelude/bwv_883/musicxml_cleaned.musicxml,Prelude_bwv_883


In [45]:
song = df.iloc[147]
print(song)
perform_row_to_annotations(song)

author                                                                  Bach
folder                                                  Bach/Prelude/bwv_883
midi2midi_alignment_path    Bach/Prelude/bwv_883/Herbert02_infer_corresp.txt
performed_midi_path                       Bach/Prelude/bwv_883/Herbert02.mid
performer                                                          Herbert02
score2midi_alignment          Bach/Prelude/bwv_883/Herbert02_infer_match.txt
score_midi_path                        Bach/Prelude/bwv_883/midi_cleaned.mid
score_xml_path                Bach/Prelude/bwv_883/musicxml_cleaned.musicxml
title                                                        Prelude_bwv_883
Name: 147, dtype: object
Processing opus Bach/Prelude/bwv_883
missing corrispondency at quant time 0.0
missing corrispondency at quant time 42.5
ind 1
filling index 0 with number 0
annotations cleaned:  {0: 'dbW', 0.516026: 'b', 2.053955: 'b', 3.550933333333333: 'db', 5.078: 'b', 6.603105: 'b', 8.024045000

'True'

## Clean for the maestro (only if aligned)

In [44]:
df = pd.read_pickle("performance_dataframe_v2(bach).pkl")
df = df[df["maestro_path"].notna()]

In [45]:
def delete_cleaned_ann(row):
    old_ann = Path(row.folder,row.performer + "_ann_cleaned.txt")
    if old_ann.exists():
        old_ann.unlink()
        
df.apply(delete_cleaned_ann,axis=1)

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
136    None
137    None
138    None
139    None
140    None
141    None
142    None
143    None
145    None
146    None
149    None
150    None
152    None
153    None
154    None
157    None
158    None
159    None
160    None
161    None
162    None
163    None
164    None
165    None
166    None
167    None
168    None
169    None
170    None
171    None
Length: 150, dtype: object